In [1]:
# %% [markdown]
# # 04 — Reporting Summary
# 
# Un notebook „all‑in‑one” care **adună rezultatele** generate în etapele anterioare și
# creează un fișier Markdown (`performance_report.md`) în `results/reports/` 
# se poate face conversie la PDF/HTML cu `pandoc` sau `jupyter nbconvert`.
# 
# ## Conținutul raportului:
# 1. Context & obiective
# 2. Rezultate test statistice (paired t, Wilcoxon, efect + CI)
# 3. Rezumat OLS & ANOVA
# 4. Curbe de putere & lățime CI
# 5. Performanță ML (RF & XGB) + grafice importanță
# 6. Concluzii & recomandări
# 
# ---
# %%
# 1️ Imports & paths
from pathlib import Path
import pandas as pd
import datetime as dt


BASE = Path("D:/Disertatie/1.database_performance_analysis").resolve()
TAB  = BASE / "results" / "tables"
FIG  = BASE / "results" / "figures"
REPORT_DIR = BASE / "results" / "reports"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

# Helper to embed image in markdown 
def md_img(path, title="", width=500):
    rel = path.relative_to(BASE)
    return f'![{title}]({rel.as_posix()}){{width="{width}px"}}'

# %%
# 2️ Load key tables
stat_sum   = pd.read_csv(TAB/"statistical_summary.csv")
ols_vs_rlm = pd.read_csv(TAB/"coef_ols_vs_rlm.csv")
anova_path = TAB/"anova_index_shard.csv"
if anova_path.exists():
    anova_tbl = pd.read_csv(anova_path)
ml_metrics = pd.read_csv(TAB/"ml_cv_metrics.csv")

# %%
# 3️ Build Markdown report string
now = dt.datetime.now().strftime("%Y-%m-%d %H:%M")
md = []
md.append(f"# Database Query Performance Report\n\n_Generated: {now}_\n")
md.append("## 1. Obiectiv\nAnaliza comparativă a timpului de execuție pentru interogări cu/ fără index și shard‑uri, folosind statistici clasice și modele de Machine‑Learning.")

md.append("## 2. Rezultate statistice clasice\n")
md.append(stat_sum.round(3).to_markdown(index=False))
md.append("\n### Bootstrap Distributions\n")
md.append(md_img(FIG/"statistical"/"bootstrap_index.png", "Index bootstrap"))
md.append(md_img(FIG/"statistical"/"bootstrap_shards.png", "Shards bootstrap"))

md.append("\n## 3. Modele liniare & ANOVA\n")
md.append("### Coeficienți OLS vs Robuști\n")
md.append(ols_vs_rlm.round(3).to_markdown())
if anova_path.exists():
    md.append("\n### ANOVA 2×2 (Index × Shard)\n")
    md.append(anova_tbl.round(3).to_markdown())

md.append("\n## 4. Putere & Learning Curves\n")
md.append(md_img(FIG/"power"/"power_curve.png", "Power curve"))
md.append(md_img(FIG/"power"/"ci_learning_curve.png", "CI learning curve"))

md.append("\n## 5. Modele ML\n")
md.append(ml_metrics.round(3).to_markdown(index=False))
md.append(md_img(FIG/"ml"/"rf_feature_importance.png", "RF importance", 400))
if (FIG/"ml"/"xgb_feature_importance.png").exists():
    md.append(md_img(FIG/"ml"/"xgb_feature_importance.png", "XGB importance", 400))

md.append("\n## 6. Concluzii\n* Indexarea reduce timpul mediu cu **{:+.1f} ms** și are un efect de mărime _d_ ≈ {stat_sum.loc[0,'cohen_d']:.2f}.\n* Sharding oferă un câștig suplimentar mediu de **{stat_sum.loc[1,'mean_diff']:.1f} ms**.\n* Modelele ML confirmă că `documents_returned` și timpul *fără* index sunt cei mai buni predictori.\n")

report_path = REPORT_DIR/"performance_report.md"
report_path.write_text("\n\n".join(md), encoding="utf-8")
print(" Markdown report generated →", report_path)

 Markdown report generated → D:\Disertatie\1.database_performance_analysis\results\reports\performance_report.md
